# Forest degradiaton detection using Deep Learning Classifaction in CSC's Puhti supercomputer
You can get acces to CSC's services from https://research.csc.fi/

Import neccessary libraries

In [4]:
import os, time
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import rasterio
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt

### Set paths according to your folder structure

In [5]:
base_folder = '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa'
csv_path = os.path.join(base_folder,'uusimaa_signatures_and_features_max_VH.csv')
model_path = os.path.join(base_folder,'uusimaa_model.pkl')

### Load features and labels of the training data

In [6]:
def load_signatures(sig_csv_path, sig_datatype=np.int32):
    """
    Extracts features and class labels from a signature CSV
    Parameters
    ----------
    sig_csv_path : str
        The path to the csv
    sig_datatype : dtype, optional
        The type of pixel data in the signature CSV. Defaults to np.int32

    Returns
    -------
    features : array_like
        a numpy array of the shape (feature_count, sample_count)
    class_labels : array_like of int
        a 1d numpy array of class labels corresponding to the samples in features.

    """
    data = np.genfromtxt(sig_csv_path, delimiter=",", dtype=sig_datatype).T
    return (data[1:, :].T, data[0, :])


In [7]:
features, labels = load_signatures(csv_path,np.float64)
features.shape

(31245, 27)

### Set again some paths

In [8]:
#Set working directory and input/output file names.

data_folder = "/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa"

results_folder = "/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/deep_classification_results"

inputImage = '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_max_VH_training.tif'
labels_dataset = labels

# Outputs of the model
# Saved model and its weights
fullyConnectedModel = os.path.join(results_folder,'fullyConnectedModel_forestloss.json')
fullyConnectedWeights = os.path.join(results_folder,'fullyConnectedWeights_forestloss.h5')
# Predicted .tif image
fullyConnectedImageCropped = os.path.join(results_folder,'uusimaa_fullyConnected.tif')
autokeras =  os.path.join(results_folder,'uusimaa_autokeras.tif')

## Part1: Classification using a user-definded neural network architecture
### Source for the three following functions https://github.com/csc-training/geocomputing/blob/master/machineLearning/03_deep/07_deepClassification.py

In [ ]:
# Train the fully connected model and save it.
# Credits: https://github.com/csc-training/geocomputing/blob/master/machineLearning/03_deep/07_deepClassification.py
def trainModel(x_train, y_train):
    start_time = time.time()    

    # Initializing a sequential model
    model = models.Sequential()
    # adding the first layer containing 64 perceptrons. 27 is representing the number of bands used for training
    model.add(layers.Dense(64, activation='relu', input_shape=(27,)))
    # add the first dropout layer
    model.add(layers.Dropout(rate=0.2))
    # adding more layers to the model
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(16, activation='relu'))
    # adding the output layer to the model, note:
	# - the activation is 'softmax', should be used for multi-class classification models
	# - size=3, for 3 classes
    model.add(layers.Dense(4, activation='softmax'))
    
	# Compile the model, using:
	# - Adam optimizer, often used, but could be some other optimizer too.
	# -- learning_rate is 0.001
	# - categorical_crossentropy loss function (should be used with multi-class classification)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Use one-hot-encoding to define a class for each pixel
    y_train_categorical = to_categorical(y_train)


    # Train the model
    model.fit(x_train,  y_train_categorical , epochs=2, batch_size=128, verbose=2)	
    
     # Save the model to disk
    # Serialize the model to JSON
    model_json = model.to_json()
    with open(fullyConnectedModel, "w") as json_file:
        json_file.write(model_json)
    # Serialize weights to HDF5
    model.save_weights(fullyConnectedWeights)
    print('Saved model to disk:  \nModel: ', fullyConnectedModel, '\nWeights: ',  fullyConnectedWeights)
    print('Model training took: ', round((time.time() - start_time), 0), ' seconds')
    return model


In [ ]:
# Predict on test data and see the model accuracy
def estimateModel(trained_model, x_test, y_test):
    
    print(y_test)

    
	# Encode the test data labels
    y_test_categorical = to_categorical(y_test)
    
    #print(y_test_categorical)
	
    # Evaluate the performance of the model by the data, it has never seen
	# verbose=0 avoids printing to output a lot of unclear text (good in Puhti)
    test_loss, test_acc = trained_model.evaluate(x_test, y_test_categorical, verbose=0)
    print('Test accuracy:', test_acc)
    
	# Calculate confusion matrix and classification report as we did with shallow classifier.
	# Use scikit-learn functions for that.
	# First predict for the x_test
    test_prediction = trained_model.predict(x_test)	
	# The model returns a 2D array, with:
	# - each row representing one pixel.
	# - each column representing the probablity of this pixel representing each category	
    print ('Test prediction dataframe shape, original 2D: ', test_prediction.shape) 	
	
	# Find which class was most likely for each pixel and select only that class for the output.
	# Output is 1D array, with the most likely class index given for each pixel.
	# Argmax returns the indices of the maximum values 
    predicted_classes = np.argmax(test_prediction,axis=1)
    print ('Test prediction dataframe shape, after argmax, 1D: ', predicted_classes.shape) 	

    print('Confusion matrix: \n', confusion_matrix(y_test, predicted_classes))
    print('Classification report: \n', classification_report(y_test, predicted_classes))

In [ ]:

# Predict on whole image and save it as .tif file
# Otherwise exactly the same as with shallow classifiers, but:
# - Load the model from a file.
# - argmax is used for the prediction results.
# - Data type is changed to in8, keras returns int64, which GDAL does not support.
def predictImage(predictedImagePath, predictImage):
    # Load json and create model
    json_file = open(fullyConnectedModel, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # Load weights into new model
    loaded_model.load_weights(fullyConnectedWeights)
    print("Loaded model from disk")
	
    # Read the satellite image
    with rasterio.open(predictImage, 'r') as image_dataset:
        start_time = time.time()    
        
        #Reshape data to 1D as we did before model training
        image_data = image_dataset.read()
        image_data2 = np.transpose(image_data, (1, 2, 0))
        pixels = image_data2.reshape(-1, 27)
        
        # Predict for all pixels
        prediction = loaded_model.predict(pixels)
        print ('Prediction dataframe shape, original 2D: ', prediction.shape) 	
		# Find the most likely class for each pixel.
        predicted_classes = np.argmax(prediction,axis=1)
        print ('Prediction dataframe shape, after argmax, 1D: ', predicted_classes.shape) 	
		  
        # Reshape back to 2D as in original raster image
        prediction2D = np.reshape(predicted_classes, (image_dataset.meta['height'], image_dataset.meta['width']))
        print('Prediction shape in 2D: ', prediction2D.shape)
		
		# Change data type to int8
        predicted2D_int8 = np.int8(prediction2D)
		
		# Save the results as .tif file.
		# Copy the coordinate system information, image size and other metadata from the satellite image 
        outputMeta = image_dataset.meta
		# Change the number of bands and data type.
        outputMeta.update(count=1, dtype='int8')
        # Writing the image on the disk
        with rasterio.open(predictedImagePath, 'w', **outputMeta) as dst:
            dst.write(predicted2D_int8, 1)
        
        print('Predicting took: ', round((time.time() - start_time), 0), ' seconds')

In [ ]:
def main():
    # Read the input datasets with Rasterio
    #labels_dataset = rasterio.open(labelsImage)
    #image_dataset = rasterio.open(inputImage)  
    
	# Prepare data for the model
    # input_image, input_labels = prepareData(image_dataset, labels_dataset)
	# Divide the data to test and training datasets
    features, labels = load_signatures(csv_path,np.float64)
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=63)
    print(x_train.shape)
    print(y_train.shape)

    # Fit and predict the fully connected deep learning model on the data. Outputs a .tif image with the predicted classification.	
    print("FullyConnected")
    fullyConnectedModel = trainModel(x_train, y_train)	
    estimateModel(fullyConnectedModel, x_test, y_test)
    # Predict image for a small training area first
    predictImage(fullyConnectedImageCropped, inputImage)
    # Predict image then for the whole uusimaa
    fullyConnectedImage = os.path.join(results_folder,'whole_uusimaa_fullyConnected.tif')
    predictImage(fullyConnectedImage, '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/UM_max_VH.tif')
    
    

In [ ]:
if __name__ == '__main__':
    ### This part just runs the main method and times it
    print("Script started!")
    start = time.time()
    main()
    end = time.time()
    print("Script completed in " + str(round((end - start),0)) + " seconds")

Script started!
(18747, 27)
(18747,)
FullyConnected


2022-12-23 23:18:00.402848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 23:18:02.882247: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30976 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:61:00.0, compute capability: 7.0


Epoch 1/2
147/147 - 4s - loss: 1.1443 - accuracy: 0.4368 - 4s/epoch - 27ms/step
Epoch 2/2
147/147 - 0s - loss: 0.8129 - accuracy: 0.5895 - 202ms/epoch - 1ms/step
Saved model to disk:  
Model:  /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/deep_classification_results/fullyConnectedModel_forestloss.json 
Weights:  /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/deep_classification_results/fullyConnectedWeights_forestloss.h5
Model training took:  8.0  seconds
[3. 2. 2. ... 1. 3. 3.]
Test accuracy: 0.6488237977027893
391/391 [==============================] - 0s 797us/step
Test prediction dataframe shape, original 2D:  (12498, 4)
Test prediction dataframe shape, after argmax, 1D:  (12498,)
Confusion matrix: 
 [[3101  835   73]
 [2372 1290   99]
 [ 665  345 3718]]
Classification report: 
               precision    recall  f1-score   support

         1.0       0.51      0.77      0.61      4009
         2.0       0.52      0.34      0.41      3761
         3.0       0.96     

## Part2: Classification using the Autokeras pipeline optimizer 

Info: https://autokeras.com/


Autokeras can be installed on Puhti like this:

In [84]:
!pip install autokeras --user

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.4/162.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 229.6 MB/s eta 0:00:00


In [9]:
import autokeras as ak

###  Split the training data into a training and a test set 
The image data can be analysed as stack of arrays where a single pixel represesents an one element of an array. \
Therefore, we can use the Structured Data Classifiers from autokeras to classify each pixel to changed or un-changed areas.

In [10]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=63)

In [ ]:
# Set the parameters for the ScruturedDataClassifier
clf = ak.StructuredDataClassifier(overwrite=False, max_trials=50, num_classes=3,seed=14)
# Perform a so-called one-hot-encoding where the label of the pixel is expressed as a binary-array. For example the label 1 of a one pixel looks like this [0.0, 1.0, 0.0, 0.0]
y_train_categorical = to_categorical(y_train)
# Use the autokeras.StructuredDataClassifier.fit() -function to perform the pipeline optimization
clf.fit(x_train, y_train_categorical, epochs=50)

In [ ]:
# Estimate the model using the test data
estimateModel(clf, x_test, y_test)
print(clf.evaluate(x_test, to_categorical(y_test)))

In [ ]:
# Analyse the exported model with the summary() -function
model = clf.export_model()
model.summary()
print(x_train.dtype)

# Save the model to the disk
try:
    model.save(os.path.join(results_folder,"model_autokeras_2512"), save_format="tf")
except Exception:
    model.save(os.path.join(results_folder,"model_autokeras2512.h5"))

In [11]:
# Saved models can be loaded like this:
from tensorflow.keras.models import load_model
loaded_model = load_model(os.path.join(results_folder,"model_autokeras_2512"), custom_objects=ak.CUSTOM_OBJECTS)

2022-12-24 15:24:49.671780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-24 15:24:52.119553: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30976 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:62:00.0, compute capability: 7.0


In [12]:
print(loaded_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 27)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 27)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 27)               55        
 n)                                                              
                                                                 
 dense (Dense)               (None, 1024)              28672     
                                                                 
 re_lu (ReLU)                (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800

In [14]:
estimateModel(loaded_model, x_test, y_test)
print(loaded_model.evaluate(x_test, to_categorical(y_test)))

[3. 2. 2. ... 3. 3. 3.]
Test accuracy: 0.8468555212020874
196/196 [==============================] - 0s 1ms/step
Test prediction dataframe shape, original 2D:  (6249, 4)
Test prediction dataframe shape, after argmax, 1D:  (6249,)
Confusion matrix: 
 [[1430  516   46]
 [ 265 1587   52]
 [  32   46 2275]]
Classification report: 
               precision    recall  f1-score   support

         1.0       0.83      0.72      0.77      1992
         2.0       0.74      0.83      0.78      1904
         3.0       0.96      0.97      0.96      2353

    accuracy                           0.85      6249
   macro avg       0.84      0.84      0.84      6249
weighted avg       0.85      0.85      0.85      6249

196/196 [==============================] - 0s 2ms/step - loss: 0.8073 - accuracy: 0.8469
[0.8073314428329468, 0.8468555212020874]


In [13]:
input_image = '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/UM_max_VH.tif'
output_file_name =  os.path.join(results_folder,'_whole_uusimaa_autokeras_.tif')

In [14]:
# Predict on whole image and save it as .tif file
def predictImage(clf, input_image,output_image_path):
    #Set file paths for input and output files
    #predictedClassesFile = outputImageBase + modelName + '.tif'
    #predictedClassesPath = os.path.join(results_folder, predictedClassesFile)
    
    # Read the satellite image
    with rasterio.open(input_image, 'r') as image_dataset:
        start_time = time.time()    
        
        #Reshape data to 1D as we did before model training
        #image_data = image_dataset.read()
        image_data = image_dataset.read()
        image_data2 = np.transpose(image_data, (1, 2, 0))
        pixels = image_data2.reshape(-1, 27)
        
        #Load the model from the saved file
        #modelFilePath = os.path.join(base_folder, ('model_' + modelName + '.sav'))
        #trained_model = load(modelFilePath)
        
        # predict the class for each pixel
        prediction = clf.predict(pixels)
        prediction = np.argmax(prediction, axis=1)
        
        # Reshape back to 2D
        print('Prediction shape in 1D: ', prediction.shape)
        prediction2D = np.reshape(prediction, (image_dataset.meta['height'], image_dataset.meta['width']))
        #prediction2D = np.reshape(prediction, ( 705, 1213))
        print('Prediction shape in 2D: ', prediction2D.shape)
        
        # Save the results as .tif file.
        # Copy the coorindate system information, image size and other metadata from the satellite image 
        outputMeta = image_dataset.meta
        # Change the number of bands and data type.
        #outputMeta.update(count=1, dtype='uint8')
        outputMeta.update(count=1, dtype='uint8', height=image_dataset.meta['height'], width=image_dataset.meta['width'])
        # Writing the image on the disk
        with rasterio.open(output_image_path, 'w', **outputMeta) as dst:
            dst.write(prediction2D, 1)
        print('Predicting took: ', round((time.time() - start_time), 1), ' seconds')


In [ ]:
predictImage(loaded_model,input_image,output_file_name)

 195532/2700000 [=>............................] - ETA: 52:43